In [1]:
from ultralytics import YOLO
# Load a model
model = YOLO('yolov8n-seg.pt')
# load a pretrained model (recommended for training)
# Train the model
results = model.train(data='coco8-seg.yaml', epochs=100, imgsz=640)

In [10]:
# tracking with track mark
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

video_path = "../../00.Data/08.YOLO/people_walk.mp4"
cap = cv2.VideoCapture(video_path)
f = open('./segment_log.txt', 'w+')

track_history = defaultdict(lambda: [])

while cap.isOpened():

    success, frame = cap.read()
    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()
        f.write('boxes : ' + str(boxes))
        f.write('track_ids : ' + str(track_ids))

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y))) # x, y center point
            if len(track) > 30: # retain 90 tracks for 90 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))

            cv2.polylines(annotated_frame, [points], isClosed=False, color=(0, 0, 200), thickness=5)

            # Display the annotated frame
            cv2.imshow("YOLOv8 Tracking", annotated_frame)

            # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

f.close()
cap.release()
cv2.destroyAllWindows()


0: 384x640 12 persons, 2 birds, 12.9ms
Speed: 4.5ms preprocess, 12.9ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 2 birds, 9.1ms
Speed: 3.7ms preprocess, 9.1ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 2 birds, 12.2ms
Speed: 8.9ms preprocess, 12.2ms inference, 3.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 2 birds, 9.5ms
Speed: 5.3ms preprocess, 9.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 2 birds, 10.8ms
Speed: 3.8ms preprocess, 10.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 2 birds, 10.2ms
Speed: 9.6ms preprocess, 10.2ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 2 birds, 7.8ms
Speed: 2.6ms preprocess, 7.8ms inference, 5.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 2 birds, 8.8ms
Speed: 5